In [1]:
import os
from DailyReport import GridBuilder, DEFAULT_GRIDS_CELL_FORMAT_CONFIG, save_doc, combine_images, get_group_value, get_zone, get_report_price_info, Instrument, get_base_instrument, get_sub_instruments, combine_report, doc2pdf
from DailyReport.template import DAILY_CHART_TEMPLATE_PATH, WEEKLY_CHART_TEMPLATE_PATH, WEEKLY_CHART_HEADER_PATH, DAILY_CHART_HEADER_PATH, DAILY_TEMPLATE_PATH
from shutil import copyfile
import math
import pandas as pd;pd.__version__

'0.24.2'

In [2]:
date = '2019-10-31'
report_folder = r'report_output'
chart_folder = 'chart_output'
data_folder = 'data_output'
daily_template_path = os.path.join(report_folder, '{0}_DAILY_CHART.docx')
daily_indicator_template_path = os.path.join(report_folder, '{0}_DAILY_INDICATOR.docx')
weekly_template_path = os.path.join(report_folder, '{0}_WEEKLY.docx')
doc_daily_path = os.path.join(report_folder, '{0}_DAILY.docx'.format(date))
doc_daily_indicator_path = os.path.join(report_folder, '{0}_DAILY_INDICATOR.docx'.format(date))
pdf_daily_path = os.path.join(report_folder, '{0}_DAILY.pdf'.format(date))
pdf_daily_indicator_path = os.path.join(report_folder, '{0}_DAILY_INDICATOR.pdf'.format(date))
doc_weekly_path = os.path.join(report_folder, '{0}_WEEKLY.docx'.format(date))
pdf_weekly_path = os.path.join(report_folder, '{0}_WEEKLY.pdf'.format(date))

In [3]:
WEEKLY_chart_list =[
    'CL ##-##',
    '6A ##-##',
    '6B ##-##',
    '6C ##-##',
    '6E ##-##',
    '6J ##-##',
    '6N ##-##',
    '6S ##-##',
    'DX ##-##',
]
DAILY_chart_list = [
    'CL ##-##',
    'AUDCAD',
    'AUDCHF',
    'AUDJPY',
    'AUDNZD',
    'AUDUSD',
    'CADCHF',
    'CADJPY',
    'CHFJPY',
    'EURAUD',
    'EURCAD',
    'EURCHF',
    'EURGBP',
    'EURJPY',
    'EURNZD',
    'EURUSD',
    'GBPAUD',
    'GBPCAD',
    'GBPCHF',
    'GBPJPY',
    'GBPNZD',
    'GBPUSD',
    'NZDCHF',
    'NZDCAD',
    'NZDJPY',
    'NZDUSD',
    'USDCAD',
    'USDCHF',
    'USDJPY'
]

In [ ]:
report_list = []
for base_instrument in WEEKLY_chart_list:
    weekly_pics_list = [
        os.path.join(chart_folder, '{0}_{1}.png'.format(base_instrument, 'WEEKLY')), 
        os.path.join(chart_folder, '{0}_{1}.png'.format(base_instrument, 'MONTHLY'))
    ]
    weekly_pic = os.path.join(report_folder, '{0}_{1}.png'.format(base_instrument, 'COMBINED'))
    combine_images(weekly_pics_list, weekly_pic)
    
    _path = weekly_template_path.format(base_instrument)
    copyfile(WEEKLY_CHART_TEMPLATE_PATH, _path)
    
    text_context = {
        'date': date,
        'instrument_name': base_instrument,
    }
    
    chart_context = {
        'weekly_monthly_chart': weekly_pic,
        'weekly_monthly_chart_width_percent': 1.5,
        'daily_chart': os.path.join(chart_folder, '{0}_{1}.png'.format(base_instrument, 'DAILY_MINUTE')),
        'daily_chart_width_percent': 1.5,
    }
    save_doc({**text_context}, {**chart_context}, {}, WEEKLY_CHART_TEMPLATE_PATH, _path)
    report_list.append(_path)
combine_report([WEEKLY_CHART_HEADER_PATH] + report_list, doc_weekly_path)
final_doc_path = os.path.abspath(doc_weekly_path)
final_pdf_path = os.path.abspath(pdf_weekly_path)
doc2pdf(final_doc_path, final_pdf_path)

In [4]:
report_list = []
for _sub in DAILY_chart_list:
    id1_daily_pics_list = [
        os.path.join(chart_folder, '{0}_{1}.png'.format(_sub, 'DAILY')), 
        os.path.join(chart_folder, '{0}_{1}.png'.format(_sub, 'ID1'))
    ]
    id1_daily_pic = os.path.join(report_folder, '{0}_{1}.png'.format(_sub, 'COMBINED'))
    combine_images(id1_daily_pics_list, id1_daily_pic)
    
    _path = daily_template_path.format(_sub)
    copyfile(DAILY_CHART_TEMPLATE_PATH, _path)
    
    text_context = {
        'date': date,
        'instrument_name': _sub,
    }
    
    chart_context = {
        'id1_daily_chart': id1_daily_pic,
        'id1_daily_chart_width_percent': 1.5,
        'id2_chart': os.path.join(chart_folder, '{0}_{1}.png'.format(_sub, 'ID2')),
        'id2_chart_width_percent': 1.5,
    }
    
    save_doc({**text_context}, {**chart_context}, {}, DAILY_CHART_TEMPLATE_PATH, _path)
    report_list.append(_path)
combine_report([DAILY_CHART_HEADER_PATH] + report_list, doc_daily_path)
final_doc_path = os.path.abspath(doc_daily_path)
final_pdf_path = os.path.abspath(pdf_daily_path)
doc2pdf(final_doc_path, final_pdf_path)

In [6]:
report_list = []
for instrument in DAILY_chart_list:    
    er_id2_path = os.path.join(data_folder, 'ER', 'ID2', f'{instrument}.csv')
    if not os.path.exists(er_id2_path):
        df = pd.DataFrame(columns=['date', 'instrument', 'value']).set_index('date')
    else:
        df = pd.read_csv(er_id2_path, names=['date', 'instrument', 'value']).set_index('date')
        df = df[df.index > date]
    er_id2_context_info = GridBuilder.create_grid_context_from_df(df, DEFAULT_GRIDS_CELL_FORMAT_CONFIG, 'p2_2')
    
    er_id1_path = os.path.join(data_folder, 'ER', 'ID1', f'{instrument}.csv')
    if not os.path.exists(er_id1_path):
        df = pd.DataFrame(columns=['date', 'instrument', 'value']).set_index('date')
    else:
        df = pd.read_csv(er_id1_path, names=['date', 'instrument', 'value']).set_index('date')
        df = df[df.index > date]
    er_id1_context_info = GridBuilder.create_grid_context_from_df(df, DEFAULT_GRIDS_CELL_FORMAT_CONFIG, 'p2_3')
    
    tr_id2_path = os.path.join(data_folder, 'TR', 'ID2', f'{instrument}.csv')
    if not os.path.exists(tr_id2_path):
        df = pd.DataFrame(columns=['date', 'instrument', 'value']).set_index('date')
    else:
        df = pd.read_csv(tr_id2_path, names=['date', 'instrument', 'value']).set_index('date')
        df = df[df.index > date]
    tr_id2_context_info = GridBuilder.create_grid_context_from_df(df, DEFAULT_GRIDS_CELL_FORMAT_CONFIG, 'p3_2')
    
    tr_id1_path = os.path.join(data_folder, 'TR', 'ID1', f'{instrument}.csv')
    if not os.path.exists(tr_id1_path):
        df = pd.DataFrame(columns=['date', 'instrument', 'value']).set_index('date')
    else:
        df = pd.read_csv(tr_id1_path, names=['date', 'instrument', 'value']).set_index('date')
        df = df[df.index > date]
    tr_id1_context_info = GridBuilder.create_grid_context_from_df(df, DEFAULT_GRIDS_CELL_FORMAT_CONFIG, 'p3_3')
    
    tr_dom_id2_path = os.path.join(data_folder, 'TR_DOM', 'ID2', f'{instrument}.csv')
    if not os.path.exists(tr_dom_id2_path):
        df = pd.DataFrame(columns=['date', 'instrument', 'value']).set_index('date')
    else:
        df = pd.read_csv(tr_dom_id2_path, names=['date', 'instrument', 'value']).set_index('date')
        df = df[df.index > date]
    tr_dom_id2_context_info = GridBuilder.create_grid_context_from_df(df, DEFAULT_GRIDS_CELL_FORMAT_CONFIG, 'p4_2')
    
    tr_dom_id1_path = os.path.join(data_folder, 'TR_DOM', 'ID1', f'{instrument}.csv')
    if not os.path.exists(tr_dom_id1_path):
        df = pd.DataFrame(columns=['date', 'instrument', 'value']).set_index('date')
    else:
        df = pd.read_csv(tr_dom_id1_path, names=['date', 'instrument', 'value']).set_index('date')
        df = df[df.index > date]
    tr_dom_id1_context_info = GridBuilder.create_grid_context_from_df(df, DEFAULT_GRIDS_CELL_FORMAT_CONFIG, 'p4_3')
    
    context = {
        'date': date,
        'instrument_name': instrument,
    }
    
    text_context = {
        **er_id2_context_info,
        **er_id1_context_info,
        **tr_id2_context_info,
        **tr_id1_context_info,
        **tr_dom_id2_context_info,
        **tr_dom_id1_context_info,
        **context
    }
    
    _path = daily_indicator_template_path.format(instrument)
    copyfile(DAILY_TEMPLATE_PATH, _path)
    
    save_doc({**text_context}, {}, {}, DAILY_TEMPLATE_PATH, _path)
    report_list.append(_path)
combine_report(report_list, doc_daily_indicator_path)
final_doc_path = os.path.abspath(doc_daily_indicator_path)
final_pdf_path = os.path.abspath(pdf_daily_indicator_path)
doc2pdf(final_doc_path, final_pdf_path)